In [243]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [4]:
data = pd.read_csv('/Volumes/MyDisk/Spring2020/Data Mining/deepsolar_tract.csv', encoding='latin1' )
data.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,education_high_school_graduate,education_less_than_high_school,education_master,education_population,education_professional_school,employed,gini_index,heating_fuel_coal_coke,heating_fuel_electricity,heating_fuel_fuel_oil_kerosene,heating_fuel_gas,heating_fuel_housing_unit_count,heating_fuel_none,heating_fuel_other,heating_fuel_solar,land_area,per_capita_income,population,population_density,poverty_family_below_poverty_level,poverty_family_count,race_asian,race_black_africa,race_indian_alaska,race_islander,race_other,race_two_more,race_white,state,total_area,unemployed,water_area,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,education_doctoral_rate,race_white_rate,race_black_africa_rate,race_indian_alaska_rate,race_asian_rate,race_islander_rate,race_other_rate,race_two_more_rate,employ_rate,poverty_family_below_poverty_level_rate,heating_fuel_gas_rate,heating_fuel_electricity_rate,heating_fuel_fuel_oil_kerosene_rate,heating_fuel_coal_coke_rate,heating_fuel_solar_rate,heating_fuel_other_rate,heating_fuel_none_rate,solar_panel_area_divided_by_area,solar_panel_area_per_capita,tile_count_residential,tile_count_nonresidential,solar_system_count_residential,solar_system_count_nonresidential,total_panel_area_residential,total_panel_area_nonresidential,median_household_income,electricity_price_residential,electricity_price_commercial,electricity_price_industrial,electricity_price_transportation,electricity_price_overall,electricity_consume_residential,electricity_consume_commercial,electricity_consume_industrial,electricity_consume_total,household_count,average_household_size,housing_unit_count,housing_unit_occupied_count,housing_unit_median_value,housing_unit_median_gross_rent,lat,lon,elevation,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,age_18_24_rate,age_25_34_rate,age_more_than_85_rate,age_75_84_rate,age_35_44_rate,age_45_54_rate,age_65_74_rate,age_55_64_rate,age_10_14_rate,age_15_17_rate,age_5_9_rate,household_type_family_rate,dropout_16_19_inschool_rate,occupation_construction_rate,occupation_public_rate,occupation_information_rate,occupation_finance_rate,occupation_education_rate,occupation_administrative_rate,occupation_manufacturing_rate,occupation_wholesale_rate,occupation_retail_rate,occupation_transportation_rate,occupation_arts_rate,occupation_agriculture_rate,occupancy_vacant_rate,occupancy_owner_rate,mortgage_with_rate,transportation_home_rate,transportation_car_alone_rate,transportation_walk_rate,transportation_carpool_rate,transportation_motorcycle_rate,transportation_bicycle_rate,transportation_public_rate,travel_time_less_than_10_rate,travel_time_10_19_rate,travel_time_20_29_rate,travel_time_30_39_rate,travel_time_40_59_rate,travel_time_60_89_rate,health_insurance_public_rate,health_insurance_none_rate,age_median,travel_time_average,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win,number_of_years_of_education,diversity,number_of_solar_system_per_household,incentive_count_residential,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0.0,0.0,0.000000,27145011200,70352.789869,Stearns County,569,1690,13,1757,336,157,4548,26,3569,0.3490,200,448,98,1720,2527,10,51,0,147.73120,26999.0,6577,44.52005,85,1911,28,19,16,0,6,88,6420,mn,154.37190,136,6.640769,0.073879,0.386324,0.371592,0.125110,0.034521,0.005717,

In [50]:
data[['tile_count','solar_system_count','county']].groupby('county').count()

,tile_count,solar_system_count
county,,
Abbeville County,6,6
Acadia Parish,12,12
Accomack County,12,12
Ada County,59,59
Adair County,22,22
...,...,...
Yuba County,14,14
Yuma County,57,57
Zapata County,3,3


In [51]:
data['tile_count'].min()

0.0

###### data.columns

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72537 entries, 0 to 72536
Columns: 169 entries, Unnamed: 0 to avg_electricity_retail_rate
dtypes: bool(2), float64(117), int64(47), object(3)
memory usage: 92.6+ MB


In [53]:
data.isnull().sum()

Unnamed: 0                                     0
tile_count                                     0
solar_system_count                             0
total_panel_area                               0
fips                                           0
average_household_income                     886
county                                         0
education_bachelor                             0
education_college                              0
education_doctoral                             0
education_high_school_graduate                 0
education_less_than_high_school                0
education_master                               0
education_population                           0
education_professional_school                  0
employed                                       0
gini_index                                   847
heating_fuel_coal_coke                         0
heating_fuel_electricity                       0
heating_fuel_fuel_oil_kerosene                 0
heating_fuel_gas    

In [54]:
data.groupby(data['county'])['total_panel_area'].mean()

county
Abbeville County      33.679973
Acadia Parish        319.494470
Accomack County       68.261343
Ada County           325.957897
Adair County         188.637733
                       ...     
Yuba County         4286.143337
Yuma County         1662.942316
Zapata County         93.654498
Zavala County         32.902217
Ziebach County         0.000000
Name: total_panel_area, Length: 1843, dtype: float64

In [55]:
cali_data = data[data['state'] == 'ca']
cali_data['tile_count'].max()

3055.0

In [127]:
def normalize(data):
    data_sqr = data*data;
    data_sum= np.sum(data_sqr)
    data_sum_sqrt = np.sqrt(data_sum)
    dataNorm = data/data_sum_sqrt
    return dataNorm

## Census Tract Level Environmental and Socioeconomic factors

### 1. Statewide Incentive parameters and Average Electricity Retail Rate

In [6]:
incentive_parameters = [

    'incentive_count_residential',
                        
                        'incentive_count_state_level',
                        'net_metering',
                        'feedin_tariff',
                        'cooperate_tax',
                        'property_tax',
                        'sales_tax',
                        'rebate',
                        'avg_electricity_retail_rate',
                        'electricity_price_residential',                                     

                       ]
len(incentive_parameters)

10

### 2. Demographic Factors

In [7]:
demographic_parameters = [
    'average_household_income',
                        'gini_index',
                        'per_capita_income',                                                    
                        'population_density',                                                   
                        'education_less_than_high_school_rate',         
'education_high_school_graduate_rate',         
'education_college_rate',                       
'education_bachelor_rate',                      
'education_master_rate',                        
'education_professional_school_rate',           
'education_doctoral_rate',                      
'race_white_rate',                              
'race_black_africa_rate',                       
'race_indian_alaska_rate',                      
'race_asian_rate',                              
'race_islander_rate',                          
'race_other_rate',                              
'race_two_more_rate',
'poverty_family_below_poverty_level_rate',      
'heating_fuel_gas_rate',                        
'heating_fuel_electricity_rate',                
'heating_fuel_fuel_oil_kerosene_rate',          
'heating_fuel_coal_coke_rate',                  
'heating_fuel_solar_rate',                      
'heating_fuel_other_rate',                      
'heating_fuel_none_rate',                       
'household_count',                                
'average_household_size',                      
'housing_unit_count',                             
'housing_unit_occupied_count',                    
'housing_unit_median_value',                   
'housing_unit_median_gross_rent',
'age_18_24_rate',                               
'age_25_34_rate',                               
'age_more_than_85_rate',                        
'age_75_84_rate',                               
'age_35_44_rate',                               
'age_45_54_rate',                               
'age_65_74_rate',                               
'age_55_64_rate',                               
'age_10_14_rate',                               
'age_15_17_rate',                               
'age_5_9_rate',                                 
'household_type_family_rate',                   
'dropout_16_19_inschool_rate',                 
'occupation_construction_rate',                 
'occupation_public_rate',                       
'occupation_information_rate',                  
'occupation_finance_rate',                      
'occupation_education_rate',                    
'occupation_administrative_rate',               
'occupation_manufacturing_rate',                
'occupation_wholesale_rate',                    
'occupation_retail_rate',                       
'occupation_transportation_rate',               
'occupation_arts_rate',                        
'occupation_agriculture_rate',                  
'occupancy_vacant_rate',                        
'occupancy_owner_rate',                         
'mortgage_with_rate',                          
'transportation_home_rate',                     
'transportation_car_alone_rate',                
'transportation_walk_rate',                     
'transportation_carpool_rate',                  
'transportation_motorcycle_rate',               
'transportation_bicycle_rate',                  
'transportation_public_rate',                   
'travel_time_less_than_10_rate',               
'travel_time_10_19_rate',                       
'travel_time_20_29_rate',                       
'travel_time_30_39_rate',                       
'travel_time_40_59_rate',                       
'travel_time_60_89_rate',                       
'health_insurance_public_rate',                 
'health_insurance_none_rate',                                 
]

len(demographic_parameters)

75

### Environmental Condition Parameters

In [233]:
envt_parameters = [
    
'heating_design_temperature',                  
'cooling_design_temperature',                  
'earth_temperature_amplitude',             
'frost_days',                                 
'air_temperature',                             
'relative_humidity',                          
'daily_solar_radiation',                       
'atmospheric_pressure',                        
'wind_speed',                                 
'earth_temperature',                           
'heating_degree_days',                         
'cooling_degree_days',
]
len(envt_parameters)

12

In [234]:
# Slicing the dataset to examine the environmental condition parameters only

envt_data = data[envt_parameters]
envt_data.head(10)

,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
0,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
3,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
4,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
5,-16.87,25.91,24.58,154.0,5.4,0.694,3.76,97.2,4.5,5.9,4820.0,1288.0
6,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
7,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
8,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
9,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0


In [235]:
# Handling missing values in out dataset

envt_data.isnull().sum()

heating_design_temperature     5802
cooling_design_temperature     5802
earth_temperature_amplitude    5802
frost_days                     5802
air_temperature                5802
relative_humidity              5802
daily_solar_radiation          5802
atmospheric_pressure           5802
wind_speed                     5802
earth_temperature              5802
heating_degree_days            5802
cooling_degree_days            5802
dtype: int64

 - The fact that all the attributes have the same number of missing values makes it look like they are having missing records from the same data objects 
     * let us take one attribute and see all the missing values if the missing records are from the same data objects

In [236]:
# In this section, 'frost_days', attribute is used to determine the missing values 
# Based on this observation - it is clear that the missing values are all from the same data objects

envt_data = data[envt_parameters]
missing_envt_data = envt_data[envt_data['frost_days'].isnull()]
envt_data_missing_val_indx = missing_envt_data.index   ## Index of the values with the missing values
missing_envt_data.head(30)

,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- The above analysis shows that certain records never contributed environmental data. It is shown from the analysis that all the missing values are from the same data records.
    * Since there are more than 5000 data records which did not provide any environmental data record at all, filling them with some value could be misleading to the analysis. 
    * Hence, the data records are dropped from the enviromental data slice.
    

In [244]:
# Dropping the environmental Data

envt_data.dropna(inplace=True)
envt_data.isnull().sum()

heating_design_temperature     0
cooling_design_temperature     0
earth_temperature_amplitude    0
frost_days                     0
air_temperature                0
relative_humidity              0
daily_solar_radiation          0
atmospheric_pressure           0
wind_speed                     0
earth_temperature              0
heating_degree_days            0
cooling_degree_days            0
dtype: int64

In [245]:
clean_envt_data = envt_data.copy()
clean_envt_data

,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
0,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
2,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
3,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
4,-16.70,25.78,24.07,154.0,5.4,0.695,3.76,97.4,4.4,5.7,4810.0,1278.0
5,-16.87,25.91,24.58,154.0,5.4,0.694,3.76,97.2,4.5,5.9,4820.0,1288.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72532,-11.09,23.96,22.47,171.0,5.8,0.534,4.54,76.9,4.8,6.8,4468.0,838.0
72533,-1.52,31.35,18.44,35.0,16.7,0.674,4.68,98.5,5.2,17.1,1598.0,2999.0
72534,-1.52,31.35,18.44,35.0,16.7,0.674,4.68,98.5,5.2,17.1,1598.0,2999.0
72535,-1.52,31.35,18.44,35.0,16.7,0.674,4.68,98.5,5.2,17.1,1598.0,2999.0


  - The environmental data is clean

### Political Views

In [189]:
pol_var = [
    
'voting_2016_dem_percentage',                     
'voting_2016_gop_percentage',                     
'voting_2016_dem_win',                            
'voting_2012_dem_percentage',                 
'voting_2012_gop_percentage',                
'voting_2012_dem_win'                           
]

len(political_parameters)

6

In [190]:
pol_data = data[pol_var]
pol_data.isnull().sum()

voting_2016_dem_percentage        3
voting_2016_gop_percentage        3
voting_2016_dem_win               0
voting_2012_dem_percentage    10554
voting_2012_gop_percentage    10554
voting_2012_dem_win               0
dtype: int64

- Voting data from 2012 seems to have a lot of missing values. Although it is possible to impute the missing values, it could otherwise be misleading due to the magnitude of missing values. Keeping the win attribute from 2012 might give us a good insight, hence it is being kept. Hence, dropping the whole attribute is the best option. 

In [191]:
pol_data.isnull().sum()

voting_2016_dem_percentage        3
voting_2016_gop_percentage        3
voting_2016_dem_win               0
voting_2012_dem_percentage    10554
voting_2012_gop_percentage    10554
voting_2012_dem_win               0
dtype: int64

In [192]:
pol_data = pol_data.drop(['voting_2012_dem_percentage',                 
'voting_2012_gop_percentage'], axis = 1)
pol_data

,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_win
0,0.323796,0.602874,False,False
1,0.323796,0.602874,False,False
2,0.323796,0.602874,False,False
3,0.323796,0.602874,False,False
4,0.323796,0.602874,False,False
...,...,...,...,...
72532,0.389874,0.524727,False,False
72533,0.194085,0.771059,False,False
72534,0.194085,0.771059,False,False
72535,0.194085,0.771059,False,False


In [193]:
# Filling the missing vlaues with the mean of the values and checking for any other missing values

mean = pol_data['voting_2016_dem_percentage'].mean()
mean1 = pol_data['voting_2016_gop_percentage'].mean()
pol_data['voting_2016_dem_percentage'].fillna(mean, inplace=True)
pol_data['voting_2016_gop_percentage'].fillna(mean1, inplace=True)
pol_data.isnull().sum()



voting_2016_dem_percentage    0
voting_2016_gop_percentage    0
voting_2016_dem_win           0
voting_2012_dem_win           0
dtype: int64

In [195]:
# Mapping True and False values of our attributes into 1 and 0 respectively
vot_map = {True: 1, False: 0}

pol_data['voting_2016_dem_win'] = pol_data['voting_2016_dem_win'].map(vot_map)
pol_data['voting_2012_dem_win'] = pol_data['voting_2016_dem_win'].map(vot_map)
pol_data.head()

,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_win
0,0.323796,0.602874,0,0
1,0.323796,0.602874,0,0
2,0.323796,0.602874,0,0
3,0.323796,0.602874,0,0
4,0.323796,0.602874,0,0


In [196]:
clean_pol_data = pol_data.copy()
clean_pol_data

,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_win
0,0.323796,0.602874,0,0
1,0.323796,0.602874,0,0
2,0.323796,0.602874,0,0
3,0.323796,0.602874,0,0
4,0.323796,0.602874,0,0
...,...,...,...,...
72532,0.389874,0.524727,0,0
72533,0.194085,0.771059,0,0
72534,0.194085,0.771059,0,0
72535,0.194085,0.771059,0,0


 - The political views data is clean